In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import gensim
import scipy
import spacy
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from scipy.stats import skew, kurtosis
import gensim
from gensim.models import Word2Vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS

from tqdm import tqdm_notebook
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/applications-of-deep-learningwustl-fall-2019/train.csv')
df_test = pd.read_csv('../input/applications-of-deep-learningwustl-fall-2019/test.csv')
df = df.dropna(how="any").reset_index(drop=True)

In [ ]:
print(df.groupby("same_source")['id'].count())
df.groupby("same_source")['id'].count().plot.bar()

In [ ]:
train_size=df.shape[0]
test_size=df_test.shape[0]
df.drop(['id'], axis=1, inplace=True)
id_test=df_test['id']
df_test.drop(['id'], axis=1, inplace=True)
df.head()

In [ ]:
df_all=pd.concat([df[['sent1','sent2']],df_test],axis=0)

In [ ]:
#tf-idf based on words
tfidf_vect = TfidfVectorizer(analyzer='word',min_df=3,token_pattern=r'\w{1,}',ngram_range=(1,2),max_features=5000)
tfidf_vect.fit(pd.concat((df_all['sent1'],df_all['sent2'])).unique())
trainq1_trans = tfidf_vect.transform(df_all['sent1'].values)
trainq2_trans = tfidf_vect.transform(df_all['sent2'].values)
fs_tfidf_word_all = scipy.sparse.hstack((trainq1_trans,trainq2_trans))

In [ ]:
fs_tfidf_word_all.shape

In [ ]:
#tf-idf based on characters
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char',min_df=3, token_pattern=r'\w{1,}',ngram_range=(1,2), max_features=5000)
tfidf_vect_ngram_chars.fit(pd.concat((df_all['sent1'],df_all['sent2'])).unique())
trainq1_trans = tfidf_vect_ngram_chars.transform(df_all['sent1'].values)
trainq2_trans = tfidf_vect_ngram_chars.transform(df_all['sent2'].values)
fs_tfidf_char_all = scipy.sparse.hstack((trainq1_trans,trainq2_trans))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [ ]:
X = scipy.sparse.hstack((fs_tfidf_char_all,fs_tfidf_word_all))
print(X.shape)

In [ ]:
y = df.loc[:, df.columns == 'same_source']
df_train, Test = train_test_split(X,train_size=train_size, random_state=42,shuffle=False)
X_train,X_val,y_train,y_val=train_test_split(df_train,y,test_size=0.05, random_state=717,shuffle=True)
#df_train, Test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
#X_train,X_val,y_train,y_val=train_test_split(df_train,y,test_size=1000, random_state=42,shuffle=False)

In [ ]:
model = xgb.XGBClassifier(max_depth=40, n_estimators=600, learning_rate=0.06, 
                          colsample_bytree=.7, gamma=0, reg_alpha=5, objective='binary:logistic', 
                          verbosity=2, subsample=0.8).fit(X_train, y_train.values.ravel(),
                                                               eval_set=[(X_train,y_train),(X_val,y_val)],
                                                               early_stopping_rounds=15) 
prediction = model.predict_proba(X_val)
#cm = confusion_matrix(y_val, prediction)  
#print(cm)  
print('Logloss', log_loss(y_val, prediction))
#print(classification_report(y_val, prediction))

In [ ]:
pred=model.predict_proba(Test)[:,1]
submission = pd.DataFrame()
submission['Id'] = id_test
submission['same_source'] = pred
submission.to_csv('output.csv', index = False)